In [32]:
import os
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision.utils import make_grid
from torchsummary import summary
from torchvision import transforms
import torch.nn.functional as F


# load pretrained network

load and store models

In [17]:
# dir(models)

alexnet = models.alexnet(weights="IMAGENET1K_V1")
efficientnet = models.efficientnet_b7(weights="IMAGENET1K_V1")
vgg_19 = models.vgg19(weights='IMAGENET1K_V1')
# inception_v3 = models.inception_v3(weights="IMAGENET1K_V1")
# resnet = models.resnet50(weights="IMAGENET1K_V2")

# model = selected_models['resnet']
# [model.state_dict()[key].shape for key in model.state_dict().keys()]

selected_models = {'alexnet':alexnet, 'efficientnet_b7':efficientnet, 'vgg_19':vgg_19}


# Modifying NN

In [18]:
class change_final_layer(nn.Module):
    def __init__(self, originalModel):
        super(change_final_layer, self).__init__()
        self.features = nn.Sequential(*list(originalModel.features))
        # print(originalModel.fc.in_features)
        last_layer = [key for key in model.state_dict().keys()][-2]
        last_layer_size = model.state_dict()[last_layer].shape[-1]
        self.classifier = nn.Sequential(*list(originalModel.classifier)[:-1], nn.Linear(last_layer_size, 15))
    
    def forward(self, x):
        return x




In [20]:
modified_models = {}
for model_name, model in selected_models.items():
    modified_model = change_final_layer(model)
    modified_models[model_name] = modified_model


# load data

In [48]:
preprocess_image = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406], 
    std=[0.229, 0.224, 0.225])
])

In [ ]:
device = torch.device(
    'cuda' if torch.cuda.is_available() else{
        'mps' if torch.backends.mps.is_available() else 'cpu'
    }
)
device

In [61]:
df = pd.read_csv("./data/data.csv") # path to images
y = F.one_hot(torch.as_tensor(df['label'].values), num_classes=15)
image_paths = df['path']
images = []

for path in image_paths:
    try:
        img = Image.open(path)
        images.append(preprocess_image(img.convert('RGB')))
    except Exception as e:
        print(f'image {path} cannot be loaded')

X = torch.stack(images)
print(X.shape)

image .\data\images\table_tennis\00000533.ashx cannot be loaded
image .\data\images\table_tennis\00000726-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000744-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000761-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000765-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000768-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000778-checkpoint.jpg cannot be loaded
image .\data\images\table_tennis\00000779-checkpoint.jpg cannot be loaded
torch.Size([10303, 3, 224, 224])


In [62]:
def get_batch(data, batch_size, iteration):
    start = batch_size * iteration
    end = (batch_size) * (iteration + 1)
    return data[start:end, :, :, :]

batch = get_batch(X, 50, 0)
print(batch.shape)

torch.Size([50, 3, 224, 224])
